In [19]:
## Tools creation
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun, Tool
from langchain_community.utilities import ArxivAPIWrapper, WikipediaAPIWrapper

In [ ]:
api_wrapper_wiki = WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250) # will use wikipedia api to conduct searches and fetch page summaries
toolWiki = WikipediaQueryRun(api_wrapper=api_wrapper_wiki) 


In [7]:
toolWiki.name

'wikipedia'

In [8]:
api_wrapper_arxiv = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=250) 
toolArxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)


In [9]:
toolArxiv.name

'arxiv'

In [10]:
tools = [toolArxiv, toolWiki]


In [12]:
## Custom tools (RAG tool)
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [13]:
loader = WebBaseLoader("https://docs.smith.langchain.com/") # base path about website document
docs = loader.load()
documents  = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 200).split_documents(docs)
vectorDB = FAISS.from_documents(documents, OllamaEmbeddings(model="gemma2:2b"))
retriver = vectorDB.as_retriever()
retriver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001535157AD90>)

In [17]:
# convert retriver into tool 
from langchain.tools.retriever import create_retriever_tool

retriever_tool = create_retriever_tool(retriever=retriver, name="langsmith search",
                                        description="Search any information about langsmith")
retriever_tool.name

'langsmith search'

In [70]:
tools = [toolArxiv,toolWiki, retriever_tool ]

In [71]:
tools

[ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=250)),
 WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Admin\\Desktop\\GenAI Krishna naik\\Langchain Basics\\.venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 Tool(name='langsmith search', description='Search any information about langsmith', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x000001534242C540>, retriever=VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAIS

## Executing tools
we use the agents to execute the tools

In [72]:
# LLM model
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
## LLM model
llm = ChatGroq(model="Llama3-8b-8192")


In [73]:
## Prompt template
from langchain import hub 
prompt = hub.pull("hwchase17/openai-functions-agent")

In [74]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [75]:
from langchain.agents import create_tool_calling_agent
agent=create_tool_calling_agent(llm = llm, tools=tools, prompt=prompt)

In [76]:
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a1655024

In [77]:
## Agent executors

from langchain.agents import AgentExecutor
agentExecute=AgentExecutor(agent= agent, tools=tools, verbose= True)

In [78]:
agentExecute

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: message_formatter(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwchase17', 'lc_

In [79]:
agentExecute.invoke({"input":"tell me about langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith search` with `{'query': 'langsmith'}`


Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first trace​Tracing to LangSmith for LangChain usersT

{'input': 'tell me about langsmith',
 'output': 'It seems like Langsmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Langsmith provides multiple ways to log traces to their platform, which can be useful for debugging and monitoring purposes.\n\nIt also seems like Langsmith is compatible with LangChain, which is a tool for building LLM applications. However, Langsmith can be used independently of LangChain.\n\nIf you have any specific questions about Langsmith or how to use it, feel free to ask!'}

In [81]:
agentExecute.invoke({"input":"What is the machine learnig?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': 'machine learning'}`


Published: 2019-09-08
Title: Lecture Notes: Optimization for Machine Learning
Authors: Elad Hazan
Summary: Lecture notes on optimization for machine learning, derived from a course at
Princeton University and tutorials given in MLSS, Buenos Aires, as
Invoking: `wikipedia` with `{'query': 'Machine learning'}`


Page: Machine learning
Summary: Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data and thus perform tasks withBased on the results from the previous tool calls, I can provide a response directly without using a tool.

Machine learning is a field of study in artificial intelligence that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. It involves the use of optimization techniques, s

{'input': 'What is the machine learnig?',
 'output': 'Based on the results from the previous tool calls, I can provide a response directly without using a tool.\n\nMachine learning is a field of study in artificial intelligence that focuses on developing and studying statistical algorithms that can learn from data and generalize to unseen data. It involves the use of optimization techniques, such as gradient descent, to adjust the parameters of a model based on the data it has seen.'}